In [1]:
# load important packages
from __future__ import print_function, division
import numpy as np
import nibabel as nib
%matplotlib
import matplotlib.pyplot as plt
import re
import scipy.stats
from scipy.stats import gamma

Using matplotlib backend: MacOSX


In [2]:
from stimuli import events2neural

In [3]:
# %load subject_class.py
from __future__ import print_function, division
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

class subject(object):
    """try to organize data based on subject ex sub001"""
    def __init__(self,sub):
        # subject info ex: 'sub001'
        self.sub_id = sub

        # BOLD folder for this subject
        sub_BOLD_path = "../data/ds105/%s/BOLD/" %self.sub_id

        # anatomy folder for this subject
        sub_anatomy_file = "../data/ds105/%s/anatomy/highres001_brain.nii.gz" %self.sub_id
        
        # runfile_list: ['task001_run001', 'task001_run002'......]
        runfile_list = ['task001_run'+ i+ '.txt' for i in ['001','002','003','004','005','006','007','008','009','010','011','012']]
        runlist = ['run'+ i for i in ['001','002','003','004','005','006','007','008','009','010','011','012']]
        
        # TR info:
        self.TR = 2.5
        
        # load high resolution brain structure for this subject
        self.brain_img = nib.load(sub_anatomy_file)
        
        # load all bold image file for this subject
        self.run_img_result = {}
        for i in runlist:
            self.run_img_result[self.sub_id + '_' + i] = nib.load(sub_BOLD_path+'task001_'+i+'/bold.nii.gz')
        
        # all run keys:
        self.run_keys = self.run_img_result.keys()

        # shape of the BOLD data:
        self.BOLD_shape = self.run_img_result[self.run_keys[1]].shape
        
        # conditions setting: which condition is for which category
        self.condition_key_file = open("../data/ds105/models/model001/condition_key.txt")
        condition_list = self.condition_key_file.readlines()
        condition = re.compile(r'(cond\d+) (\w+)')
        result = {}
        for item in condition_list:
            for match in condition.finditer(item):
                result[match.group(1)] = match.group(2)
        self.condition_key = result
        
        # condition files for each objects for each run
        sub_condition_path = "../data/ds105/%s/model/model001/onsets/" %self.sub_id
        self.conditions = {}
        for i in runfile_list:
            for j in self.condition_key.keys():
                self.conditions[i[8:14]+'-'+self.condition_key[j]] = sub_condition_path + i[:-4]+'/'+j+'.txt'
                   
                



In [5]:
pwd

u'/Users/edithho/Google Drive/cal/2015 fall/stat159/project-zeta/experimental'

In [4]:
sub1 = subject("sub001") # Loop this for other subjects...LATER

IOError: [Errno 2] No such file or directory: '../data/ds105/sub001/anatomy/highres001_brain.nii.gz'

In [ ]:
sub1.BOLD_shape

In [ ]:
sub1.TR

In [ ]:
sub1.brain_img

In [ ]:
sub1.condition_key

In [ ]:
sub1_condition_files = sub1.conditions

In [ ]:
sub1_condition_files

In [ ]:
sub1.sub_id

In [ ]:
# %load get_object_neural.py

def get_object_neural(sub_ID ,condition_dict, object_name, TR, n_vox, check = 0, ):
    """
    get object neural array for specific object from all runs, odd runs or even runs
    
    parameters
    ----------
    condition_dict: dictionary of all condition files
    object_name: desired object name: 'house','scrambledpix','cat','shoe','bottle','scissors','chair','face'
    TR: TR for fMRI
    n_vox: time course of fMRI
    check: 1= odd runs, 2=even runs, others = all runs
    
    output
    -------
    a dictionary of run name : (neural array, path of the bold image) for that run
    ex:
    
    {
    'run001-bottle-neural': (array([ 0.,  0.,  0.,  0.,  0., ...]), bold path)
    'run002-bottle-neural': (array([ 0.,  0.,  0.,  0.,  0., ...]), bold path)
    }
    
    
    """
    
    condition_dict_key = condition_dict.keys()
    result = {}
    for i in condition_dict_key:
        if check == 1:
            if object_name in i:
                if int(i[3:6])%2 ==1:
                    result[i+'-neural'] = (events2neural(condition_dict[i], TR, n_vox), "../data/ds105/%s/BOLD/task001_"% sub_ID + i[:6]+"/bold.nii.gz")
        elif check == 2:
            if object_name in i: 
                if int(i[3:6])%2 ==0:
                    result[i+'-neural'] = (events2neural(condition_dict[i], TR, n_vox), "../data/ds105/%s/BOLD/task001_"% sub_ID + i[:6]+"/bold.nii.gz")
        else:
            if object_name in i:
                result[i+'-neural'] = (events2neural(condition_dict[i], TR, n_vox), "../data/ds105/%s/BOLD/task001_"%sub_ID + i[:6]+"/bold.nii.gz")
    
    return result




In [ ]:
cat_odd_neural = get_object_neural(sub1.sub_id, sub1.conditions,"cat" , sub1.TR, sub1.BOLD_shape[-1], check=1)
cat_even_neural = get_object_neural(sub1.sub_id, sub1.conditions,"cat" , sub1.TR, sub1.BOLD_shape[-1], check=2)
house_odd_neural = get_object_neural(sub1.sub_id, sub1.conditions,"house" , sub1.TR, sub1.BOLD_shape[-1], check=1)
house_even_neural = get_object_neural(sub1.sub_id, sub1.conditions,"house" , sub1.TR, sub1.BOLD_shape[-1], check=2)
bottle_odd_neural = get_object_neural(sub1.sub_id, sub1.conditions,"bottle" , sub1.TR, sub1.BOLD_shape[-1], check=1)
bottle_even_neural = get_object_neural(sub1.sub_id, sub1.conditions,"bottle" , sub1.TR, sub1.BOLD_shape[-1], check=2)
face_odd_neural = get_object_neural(sub1.sub_id, sub1.conditions,"face" , sub1.TR, sub1.BOLD_shape[-1], check=1)
face_even_neural = get_object_neural(sub1.sub_id, sub1.conditions,"face" , sub1.TR, sub1.BOLD_shape[-1], check=2)
scissors_odd_neural = get_object_neural(sub1.sub_id, sub1.conditions,"scissors" , sub1.TR, sub1.BOLD_shape[-1], check=1)
scissors_even_neural = get_object_neural(sub1.sub_id, sub1.conditions,"scissors" , sub1.TR, sub1.BOLD_shape[-1], check=2)
shoe_odd_neural = get_object_neural(sub1.sub_id, sub1.conditions,"shoe" , sub1.TR, sub1.BOLD_shape[-1], check=1)
shoe_even_neural = get_object_neural(sub1.sub_id, sub1.conditions,"shoe" , sub1.TR, sub1.BOLD_shape[-1], check=2)
scrambledpix_odd_neural = get_object_neural(sub1.sub_id, sub1.conditions,"scrambledpix" , sub1.TR, sub1.BOLD_shape[-1], check=1)
scrambledpix_even_neural = get_object_neural(sub1.sub_id, sub1.conditions,"scrambledpix" , sub1.TR, sub1.BOLD_shape[-1], check=2)
chair_odd_neural = get_object_neural(sub1.sub_id, sub1.conditions,"chair" , sub1.TR, sub1.BOLD_shape[-1], check=1)
chair_even_neural = get_object_neural(sub1.sub_id, sub1.conditions,"chair" , sub1.TR, sub1.BOLD_shape[-1], check=2)

neuralDict = []
for key, value in sub1.condition_key.iteritems():
    for i in xrange(1,3):
        neuralObj = get_object_neural(sub1.sub_id, sub1.conditions, value, sub1.TR, sub1.BOLD_shape[-1], check = i)
        neuralDict.append(neuralObj)
        
len(neuralDict)
#cat_odd_neural = get_object_neural(sub1.sub_id, sub1.conditions,"cat" , sub1.TR, sub1.BOLD_shape[-1], check=1)

In [ ]:
cat_odd_neural

In [ ]:
cat_odd_neural['run001-cat-neural']

In [ ]:
# %load ObtainData.py
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

def pull3D(DictionaryOfVectors):
	Three_D_Data = []
	for key, value in DictionaryOfVectors.iteritems():
		vector = value[0]
		fileLocation = value[1]
		img = nib.load(fileLocation)
		data = img.get_data()
		for i in range(0, len(vector)):
			if vector[i] == 1:
				Three_D_Data.append(data[:, :, :, i])
	return Three_D_Data

def ShowCortex(ListOf3dImages):
	for image in ListOf3dImages:
		zDimension = np.shape(image)[2]
		for i in range(0, zDimension):
			plt.imshow(image[:, :, i], cmap = "gray")
			plt.show()

# You'll have to guess the Z value from the images show in the previous function
def generateEvenlySpacedZpoints(zValue):
	return np.linspace(start = 0, stop = zValue, num = zValue)

In [ ]:
listof3D = []
for neural in neuralDict:
    listof3D.append(pull3D(neural))

print(len(listof3D))

#aa = pull3D(cat_odd_neural)

In [ ]:
len(aa)

In [6]:
condList = []
for key, value in sub1_condition_files.iteritems():
    condList.append(sub1_condition_files[key])

condList
    
run1_face_cond = sub1_condition_files['run001-face'] # Loop this
run1_face_cond

NameError: name 'sub1_condition_files' is not defined

In [30]:
run1_face_neural_prediction = events2neural(run1_face_cond, sub1.TR, sub1.BOLD_shape[-1])

In [36]:
all_tr_times = np.arange(sub1.BOLD_shape[-1]) * sub1.TR
plt.plot(all_tr_times, run1_face_neural_prediction)

In [41]:
def hrf(times):
    """ Return values for HRF at given times """
    # Gamma pdf for the peak
    peak_values = gamma.pdf(times, 6)
    # Gamma pdf for the undershoot
    undershoot_values = gamma.pdf(times, 12)
    # Combine them
    values = peak_values - 0.35 * undershoot_values
    # Scale max to 0.6
    return values / np.max(values) * 0.6

In [42]:
tr_times = np.arange(0, 30, sub1.TR) # BUT WHY 30
hrf_at_trs = hrf(tr_times) 
len(hrf_at_trs)

12

In [44]:
convolved = np.convolve(run1_face_neural_prediction, hrf_at_trs)
N = len(run1_face_neural_prediction)  # M == # of volumes == 121
M = len(hrf_at_trs)  # M == 12
len(convolved) == N + M - 1

True

In [45]:
n_to_remove = len(hrf_at_trs) - 1
convolved = convolved[:-n_to_remove]

In [47]:
plt.plot(all_tr_times, run1_face_neural_prediction)
plt.plot(all_tr_times, convolved)